In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-17 14:43:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-04-17 14:43:01 (11.4 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

### Create DataFrames to match tables

In [7]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [8]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28377689|             1|
|   28258386|             2|
|    9967574|             1|
|   25153155|             1|
|   10088068|             1|
|   45657423|             1|
|   19021463|             1|
|   41413793|             2|
|   43789873|             2|
|   12406466|             1|
|   41045019|             1|
|   35535911|             2|
|   47108763|             1|
|   48113150|             1|
|   18201417|             9|
|   26079415|             3|
|   36114891|             7|
|    3712628|             1|
|   37499901|             2|
|   47321438|             2|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates(['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0538603879|   Mr. Happy Incense|
|0963469142|         Five Crowns|
|0983124221|SioGreen iR Tankless|
|100100003X|MAGCREDIBLE® Magn...|
|104800001X|MAGCREDIBLE® Magn...|
|1094780162|Men Automatic U1 ...|
|139475891X|Stylish Triple Ti...|
|140050337X|50 Super Hook Wal...|
|140755736X|RotoStyler Rotati...|
|142391550X|Renata Battery 39...|
|1450848214|Westinghouse 3 ou...|
|148796174X|Vogue Goddess Sil...|
|157223217X|Just Labs:  Half-...|
|1590651677|Pantone Plus Seri...|
|1590651707|Pantone Plus Seri...|
|1590651715|Pantone Plus Seri...|
|1590651804|Pantone Plus Seri...|
|159065322X|PANTONE GG1504 Pl...|
|1590653254|PANTONE GG1511 Pl...|
|1590653289|PANTONE GP1501 Pl...|
+----------+--------------------+
only showing top 20 rows



In [10]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R215C9BDXTDQOW|   48881148|B00FR4YQYK|     381800308| 2015-08-31|
|R1DTPUV1J57YHA|   47882936|B00439MYYE|     921341748| 2015-08-31|
| RFAZK5EWKJWOU|   44435471|B00002N762|      56053291| 2015-08-31|
|R2XT8X0O0WS1AL|   28377689|B000QFCP1G|     595928517| 2015-08-31|
|R14GRNANKO2Y2J|   50134766|B00WRCRKOI|     417053744| 2015-08-31|
|R2BLF9VYL24LCQ|   14066511|B00NIH88EW|     275395071| 2015-08-31|
|R1GI9UW5KJ671O|   15211046|B005B9CI96|     856617815| 2015-08-31|
|R2H5CEJN863M86|   14862498|B008L0OMWI|     125102494| 2015-08-31|
| R5PPDHFOZ3SMU|   23617292|B00P9FTC6O|     523110842| 2015-08-31|
| RE1L9IENKJJ7Y|   35820485|B00K6BQEHQ|     797306964| 2015-08-31|
|R3CZ0990QC2Z0H|   47162350|B008BYQCWM|     865874404| 2015-08-31|
|R3UMMD2IO29QSP|   31884789|B003BLHTOU|     183592595| 2015-08

In [11]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R215C9BDXTDQOW|          4|            0|          0|   N|                Y|
|R1DTPUV1J57YHA|          5|            0|          0|   N|                Y|
| RFAZK5EWKJWOU|          5|            0|          0|   N|                Y|
|R2XT8X0O0WS1AL|          5|            0|          0|   N|                Y|
|R14GRNANKO2Y2J|          5|            0|          0|   N|                Y|
|R2BLF9VYL24LCQ|          5|            1|          1|   N|                Y|
|R1GI9UW5KJ671O|          5|            0|          0|   N|                Y|
|R2H5CEJN863M86|          5|            0|          1|   N|                Y|
| R5PPDHFOZ3SMU|          5|            0|          0|   N|                Y|
| RE1L9IENKJJ7Y|          1|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mod16challenge.cov3o2zuhbsx.us-east-2.rds.amazonaws.com:5432/Mod16Challenge"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

DELIVERABLE 2

In [12]:
filtered_vine_df = vine_df.filter(vine_df.total_votes>19)
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [13]:
helpful_votes_df = filtered_vine_df.withColumn('Helpful_Total_Votes',filtered_vine_df['helpful_votes']/filtered_vine_df['total_votes']).filter(('Helpful_Total_Votes>=.5') )
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|                1.0|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y| 0.9555555555555556|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y| 0.8873239436619719|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y| 0.9655172413793104|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y| 0.9142857142857143|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y| 0.9217391304347826|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y| 0.9705882352941176|
| RQCC7XNYI014B|    

In [14]:
vine_votes_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R29V4UDSS053D8|          3|           33|         38|   Y|                N|  0.868421052631579|
|R1I2D068WC37PA|          3|           32|         37|   Y|                N| 0.8648648648648649|
|R13W2U74F67QED|          5|           62|         76|   Y|                N| 0.8157894736842105|
|R2QI37XFOBKUGD|          4|           24|         33|   Y|                N| 0.7272727272727273|
|R19F60BB2DNCKN|          5|          170|        174|   Y|                N| 0.9770114942528736|
|R2PYNEWYYQDWHI|          5|           24|         27|   Y|                N| 0.8888888888888888|
|R1UEWHCPJL2XJB|          5|           70|         72|   Y|                N| 0.9722222222222222|
| RXTMMEBRYZB53|    

In [15]:
vine_votes_df.count()

266

In [18]:
vine_5_star = vine_votes_df.filter(vine_votes_df.star_rating == 5)
vine_5_star.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R13W2U74F67QED|          5|           62|         76|   Y|                N| 0.8157894736842105|
|R19F60BB2DNCKN|          5|          170|        174|   Y|                N| 0.9770114942528736|
|R2PYNEWYYQDWHI|          5|           24|         27|   Y|                N| 0.8888888888888888|
|R1UEWHCPJL2XJB|          5|           70|         72|   Y|                N| 0.9722222222222222|
| RXTMMEBRYZB53|          5|           21|         24|   Y|                N|              0.875|
|R1PXVYIYMYGUZL|          5|           21|         26|   Y|                N| 0.8076923076923077|
| RSFMJJXFZHX1C|          5|           24|         25|   Y|                N|               0.96|
|R1Z849RVP83BSJ|    

In [20]:
vine_5_star.count()

125

In [33]:
(vine_5_star.count()/ vine_votes_df.count())*100

46.99248120300752

In [28]:
non_vine_votes_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
non_vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|                1.0|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y| 0.9555555555555556|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y| 0.8873239436619719|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y| 0.9655172413793104|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y| 0.9142857142857143|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y| 0.9217391304347826|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y| 0.9705882352941176|
| RQCC7XNYI014B|    

In [29]:
non_vine_votes_df.count()

38829

In [30]:
non_vine_5_star = non_vine_votes_df.filter(non_vine_votes_df.star_rating == 5)
non_vine_5_star.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Total_Votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y| 0.9555555555555556|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y| 0.9655172413793104|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y| 0.9705882352941176|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|                1.0|
|R2KBV3A9X53ULG|          5|           39|         45|   N|                Y| 0.8666666666666667|
|R18XX6GVZ41CY1|          5|           21|         22|   N|                N| 0.9545454545454546|
|R3V8NHJWQWOU7S|          5|           27|         27|   N|                Y|                1.0|
|R16U59S8ZJGPLY|    

In [31]:
non_vine_5_star.count()

18246

In [34]:
(non_vine_5_star.count()/ non_vine_votes_df.count())*100

46.99065131731438